In [4]:
# 13/10/2020 4.11pm
import pandas as pd
import pymongo
from pandas import DataFrame
from pymongo import MongoClient
client = MongoClient("Name Id Password of Database")
db = client."NameofDatabase"

CASA = borrowers.aggregate([
{   
            "$project" :  {
                "_id":1.0,
                "Account_No":"$Account_No",
                "Date":"$Date",
                "Narration":"$Narration",
                "Dr_Amount":"$Dr_Amount",
                "Cr_Amount":"$Cr_Amount",
                "Chq/Ref_Number":"$Chq/Ref_Number",
                "Closing_Balance":"$Closing_Balance"
                        }   
}
])
Casa_Txn = DataFrame(list(CASA))
print(Casa_Txn)
Casa_Txn.to_csv(r"PATH\CASA_TXNS.csv",index=False)

import pandas as pd
import numpy as np

train=pd.read_csv('CASA_TXNS.csv')
train.rename(columns={'Account No':'Account_No'},inplace=True)
train.rename(columns={'Chq/Ref Number':'Chq/Ref_Number'},inplace=True)
train.rename(columns={'Closing Balance':'Closing_Balance'},inplace=True)

train['Dr. Amount']=train['Dr. Amount'].apply(lambda x: 0 if pd.isnull(x) else x)
train['Cr. Amount']=train['Cr. Amount'].apply(lambda x: 0 if pd.isnull(x) else x)
train['Txn_Amt'] = [-x if x!=0 else y  for x,y in zip(train['Dr. Amount'],train['Cr. Amount'])]

load=pd.read_csv('CASA_OPENING_BALANCE.csv')
load.rename(columns={'Account No':'Account_No'},inplace=True)
load.rename(columns={'Opening Balance':'Opening_Balance'},inplace=True)

load['Txn_Amt']=load['Opening_Balance'].apply(lambda x: -x if x==-x else x)
load['Date']=pd.to_datetime(load['Date'])

import datetime 
train['new_date']=pd.to_datetime(train['Date'],format='%d-%b-%y').dt.strftime('%Y-%m')
train['Date']=pd.to_datetime(train['Date'],format='%d-%b-%y').dt.strftime('%Y-%m-%d')

a=(train['Account_No']).unique()

dummy=train.groupby(['Account_No','new_date']).size().reset_index()

from dateutil.relativedelta import relativedelta

dummy['new_date'] = pd.to_datetime(dummy.new_date.astype(str),format='%Y-%m')
dummy = dummy.set_index('new_date').groupby('Account_No').resample('MS').asfreq().ffill().reset_index(level=0, drop=True)
dummy = dummy.reset_index()
dummy['new_date'] = dummy.new_date.dt.strftime('%Y-%m')
dummy = dummy.drop(0,axis=1)
dummy = dummy[['Account_No','new_date']]
dummy['Account_No'] = dummy['Account_No'].astype('int64')

dummy['new_date']=pd.to_datetime(dummy['new_date'],format='%Y-%m').dt.strftime('%Y-%m-%d')
dummy['new_date'] = dummy['new_date'].apply(lambda y:datetime.datetime.strptime(y,'%Y-%m-%d'))
dummy['new_date']=dummy['new_date'].apply(lambda x : x + relativedelta(months = 1))  

dummy['Txn_Amt']=0

dummy.rename(columns={'new_date':'Date'},inplace=True)

CASA_TXNS=train[['Account_No','Date','Txn_Amt']]

CON= pd.concat([CASA_TXNS, dummy],ignore_index=True)

CON['Date']=pd.to_datetime(CON['Date'],format='%Y-%m-%d')

new_load=load[['Account_No','Date','Txn_Amt']]

new_con=pd.concat([CON,new_load],ignore_index=True)
new_con['Date']=pd.to_datetime(new_con['Date'],format='%Y-%m-%d')

new_con.sort_index(by=['Account_No','Date'],inplace=True)

new_con['Running_bal']=new_con['Txn_Amt'].cumsum()

new_con['Prev_Txn_Date'] = new_con["Date"].shift()
new_con['Prev_Running_Bal'] = new_con["Running_bal"].shift() 


new_con['No_Of_Days']=new_con["Date"]-new_con['Prev_Txn_Date']
new_con['No_Of_Days']=new_con['No_Of_Days']/np.timedelta64(1,'D')

new_con['Mth_ID']=pd.to_datetime(new_con['Prev_Txn_Date'],format='%Y-%m-%d').dt.strftime('%Y%m')
new_con.to_csv(r'D:\tempdir\Titanics\stepx3.csv')


new_con['Sum_Product'] = new_con['Prev_Running_Bal']*new_con['No_Of_Days']

stepx5 = new_con.groupby(['Account_No','Mth_ID']).agg({'No_Of_Days':'sum',
                                                       'Prev_Running_Bal':'sum','Sum_Product':'sum'}).reset_index()

stepx5['Avg_Mthly_bal']=stepx5['Sum_Product']/stepx5['No_Of_Days']
stepx5['Avg_Mthly_bal']=stepx5['Avg_Mthly_bal'].round(decimals=2)
stepx5=stepx5[pd.notnull(stepx5['Avg_Mthly_bal'])]
stepx5=stepx5.drop(['Prev_Running_Bal'], axis = 1)
stepx5.head(20)

















C:\Users\Junaid\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)


,Account_No,Mth_ID,No_Of_Days,Sum_Product,Avg_Mthly_bal
0,10851232411,201104,30.0,619225.00,20640.83
1,10851232411,201105,31.0,456475.00,14725.00
2,10851232411,201106,30.0,441750.00,14725.00
3,10851232411,201107,31.0,2440141.76,78714.25
4,10851232411,201108,31.0,3955732.41,127604.27
5,10851232411,201109,30.0,3752892.51,125096.42
7,10851232494,201104,30.0,6681797.45,222726.58
8,10851232494,201105,31.0,6687037.91,215710.90
9,10851232494,201106,30.0,5415638.30,180521.28
10,10851232494,201107,31.0,5126892.91,165383.64
